In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
denver_path = "./Data/denvercrime.csv"
crime_code_path = "./Data/offense_codes.csv"
df = pd.read_csv(denver_path)
crime_code_df = pd.read_csv(crime_code_path)
event_df = pd.read_csv('./Dimensions/dimension_event.csv')
event_df = event_df[event_df['event_city'].isin(["Denver"])]

In [ ]:
event_start_lst = event_df["event_start_date"].tolist()
event_end_lst = event_df["event_end_date"].tolist()

In [ ]:
crime_code_index = []
crime_details=[]
for index, row in crime_code_df.iterrows():
    crime_code_index.append((row["OFFENSE_CODE"],row["OFFENSE_CODE_EXTENSION"]))
    crime_details.append(row["OFFENSE_TYPE_NAME"])
# Source https://www.macrotrends.net/cities/us/co/denver/crime-rate-statistics
# Denver population 2790,000 in 2019, data set has 95272 records in 2019
crime_rate_list ={"2015":6.739,"2016":6.574,"2017":6.756,"2018":7.301,"2019":3.415}

In [ ]:
def time_cov(raw_in):
    if type(raw_in)==float:
        return raw_in,raw_in,raw_in,raw_in,raw_in,raw_in
    datetime_raw = datetime.strptime(raw_in, '%m/%d/%Y %I:%M:%S %p')
    date = int(datetime_raw.strftime("%Y%m%d"))
    hour = datetime_raw.hour
    minute = datetime_raw.minute
    second = datetime_raw.second
    dow = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"][datetime_raw.weekday()]
    return date,hour,minute,second,dow,datetime_raw


In [ ]:
loc_df_list=[]
crime_df_list=[]
fact_df_list=[]

In [ ]:
for index, row in df.iterrows():
        
    s_d,s_h,s_m,s_s,s_dow,s_dt = time_cov(row["FIRST_OCCURRENCE_DATE"])
    if not(2015<=s_dt.year<=2019):
        continue
    e_d,e_h,e_m,e_s,e_dow,e_dt = time_cov(row["LAST_OCCURRENCE_DATE"])
    r_d,r_h,r_m,r_s,r_dow,r_dt = time_cov(row["REPORTED_DATE"])

    date_id = int(s_dt.strftime("%Y%m%d"))    
    
    location_id = index + 1000000
    city = "Denver"
    location = row["INCIDENT_ADDRESS"]
    neighborhood = row["NEIGHBORHOOD_ID"]
    lat = row["GEO_LAT"]
    lon = row["GEO_LON"]
    
    
    
    crime_id = index + 2000000
    crime_category_name = row["OFFENSE_CATEGORY_ID"]
    crime_type_name = row["OFFENSE_TYPE_ID"]
    crime_detail = crime_details[crime_code_index.index((row["OFFENSE_CODE"],row["OFFENSE_CODE_EXTENSION"]))]
    
    
    #Event
    event_id = np.nan
    #for idx_event, row_event in event_df.iterrows():
    for i in range(len(event_start_lst)):
        #the crime is in the event period
        if date_id>=event_start_lst[i] and date_id<=event_end_lst[i]:
            row_event = event_df.iloc[i]
            local_location = location.upper() if type(location)==str else "(｡･ω･)ﾉﾞ"
            event_location = row_event['event_location'].upper() if type(row_event['event_location'])==str else "ヾ(´･ω･｀)ﾉ"
            if local_location in event_location:
                event_id = row_event['event_key']
                break
            elif neighborhood == row_event['event_neighbourhood']:
                event_id = row_event['event_key']
            elif row_event['event_location_size'] == 'city' and np.isnan(event_id):
                event_id = row_event['event_key']
                    
    
    
    is_night = 0 if 7 <= s_h <= 18 else 1
    
    fatal_keywords=['FATAL','DEAD','DEATH','KILL','HOMICIDE','MURDER']
    is_fatal = 0
    for keyword in fatal_keywords:
        for name in [crime_category_name,crime_type_name,crime_detail]:
            if keyword in name.upper():
                is_fatal = 1
    crime_rate = crime_rate_list[str(s_dt.year)]
                
    #location_df.loc[index] = [location_id,city,location,neighborhood,lat,lon,crime_rate]
    #crime_df.loc[index] = [crime_id,crime_category_name,crime_type_name,crime_detail,s_h,s_m,s_s,e_h,e_m,e_s,r_h,r_m,r_s]
    #fact_df.loc[index] = [location_id,crime_id,date_id,event_id,row["IS_TRAFFIC"],is_fatal,is_night]

    loc_df_list.append([location_id,city,location,neighborhood,lat,lon,crime_rate])
    crime_df_list.append([crime_id,crime_category_name,crime_type_name,crime_detail,s_h,s_m,s_s,e_h,e_m,e_s,r_h,r_m,r_s])
    fact_df_list.append([location_id,crime_id,date_id,event_id,row["IS_TRAFFIC"],is_fatal,is_night])

In [ ]:
fact_df = pd.DataFrame(fact_df_list, columns=['location_Key','crime_key', 'date_key','event_key', 'is_traffic','is_fatal','is_nighttime'],dtype="Int64")
location_df = pd.DataFrame(loc_df_list, columns=['location_Key','city','location_name', 'neighborhood', 'latitude','longitude','crime_rate_percentage'])
crime_df= pd.DataFrame(crime_df_list, columns=['crime_key','crime_category_name', 'crime_type_name','crime_details',
                                 'start_hour', 'start_minute',"start_second",
                                 'end_hour', 'end_minute',"end_second",
                                 'report_hour', 'report_minute',"report_second"])

In [ ]:
location_df.head()

In [ ]:
crime_df.head()

In [ ]:
fact_df.head()

In [ ]:
df.head()

In [ ]:
fact_df.to_csv(r'.\Dimensions\fact_denver.csv',index=False)
crime_df.to_csv(r'.\Dimensions\crm_denver.csv',index=False)
location_df.to_csv(r'.\Dimensions\loc_denver.csv',index=False)